<a href="https://colab.research.google.com/github/88deeja/Explotation_des_compteurs_de_trafic/blob/main/ABAOUSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import cmath
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



# Constantes
M_sacc = 342.18
M_eau = 18
R = 8.314
# Données pour le saccharose (sacc)
Tc_sacc = 647.31  # Température critique du saccharose en K
Pc_sacc = 80.06*101325   # Pression critique du saccharose en Pascal
omega_sacc = 0.369
# Données pour l'eau (eau)
Tc_eau = 647.06   # Température critique de l'eau en K
Pc_eau = 220.50*101325   # Pression critique de l'eau en Pascal
omega_eau = 0.348



X = np.array([0.055, 0.106, 0.215, 0.32, 0.445, 0.596, 0.65, 0.76, 0.785, 0.82, 0.865, 0.815, 0.875])
x_sacc = []
for i in range(len(X)):
    frac= (X[i]/M_sacc)/(X[i]/M_sacc + (1 - X[i])/M_eau )
    x_sacc.append(frac)


x_eau = []
for j in range(len(x_sacc)):
    frac = 1 - x_sacc[j]
    x_eau.append(frac)




T_C = np.array([100.1, 100.3, 100.4, 101.2, 101.8, 103, 104, 108.2, 110, 115, 118, 107.4, 117])
T_K = []
for i in T_C:
    i = i + 273.15
    T_K.append(i)

def calculate_A(T, P, a, R):
    A = (a * P) / (R * T)**2
    return A

def calculate_B(T, P, R, b):
    B = (b * P) / (R * T)
    return B

def ac(R, Tc, Pc):
    return 0.45724 * ((R**2 * Tc**2) / Pc)

def a(R, Tc, Pc, m, T):
    return ac(R, Tc, Pc) * (1 + m * (1 - np.sqrt(T/Tc)))**2

def b(R, Tc, Pc):
    return 0.07780 * (R * Tc) / Pc


def m(omega):
    return 0.37464 + 1.54226 * omega - 0.26992 * omega**2

P = 1*101325
Z_sacc = []
for i in range(len(T_K)):
    m_value = m(omega_sacc)
    ac_value = ac(R, Tc_sacc, Pc_sacc)
    a_value = a(R, Tc_sacc, Pc_sacc, m_value , T_K[i])
    b_value = b(R, Tc_sacc, Pc_sacc)
    A = calculate_A(T_K[i], P, a_value, b_value)
    B = calculate_B(T_K[i], P, a_value, b_value)
    u  = 1 - B
    v = A - 3*B**2 - 2*B
    w = A*B - B**2 - B**3
    coefficients = [1,u, v, w]
    z = np.roots(coefficients)
    Z_sacc.append(z)
Z_reel_sacc = []
for z in Z_sacc:
    z_reel = max(z, key=lambda x: x.real)
    z_reel = np.real(z_reel)
    Z_reel_sacc.append(z_reel)




Z_eau = []
for i in range(len(T_K)):
    m_value = m(omega_eau)
    ac_value = ac(R, Tc_eau, Pc_eau)
    a_value = a(R, Tc_eau, Pc_eau, m_value , T_K[i])
    b_value = b(R, Tc_eau, Pc_eau)
    A = calculate_A(T_K[i], P, a_value, b_value)
    B = calculate_B(T_K[i], P, a_value, b_value)
    u  = 1 - B
    v = A - 3*B**2 - 2*B
    w = A*B - B**2 - B**3
    coefficients = [1,u, v, w]
    z = np.roots(coefficients)
    Z_eau.append(z)
Z_reel_eau = []
for z in Z_eau:
    z_reel = max(z, key=lambda x: x.real)
    z_reel = np.real(z_reel)
    Z_reel_eau.append(z_reel)



coef_fug_eau = []
for i in range(len(Z_reel_eau)):
    a = (Z_reel_eau[i] - 1) * np.log(P)
    coef_fug_eau.append(a)

coef_fug_sacc = []
for i in range(len(Z_reel_sacc)):
    a = (Z_reel_sacc[i] - 1) * np.log(P)
    coef_fug_sacc.append(a)

Vol_eau = []
for z, t in zip(Z_reel_eau, T_K):
    v = (z * R * t)/ P
    Vol_eau.append(v)

Vol_sacc = []
for z, t in zip(Z_reel_sacc, T_K):
    v = (z * R * t)/ P
    Vol_sacc.append(v)



ac_eau = ac(R, Tc_eau, Pc_eau)
ac_sacc = ac(R, Tc_sacc, Pc_sacc)
b_eau = b(R, Tc_eau, Pc_eau)
b_sacc = b(R, Tc_sacc, Pc_sacc)

b_m = []
for xe, xs in zip(x_eau, x_sacc):
    bm = xe * b_eau + xs * b_sacc
    b_m.append(bm)




k_eau_sacc=[]
for t, xe, xs, z, vol, phi, bm in zip(T_K, x_eau, x_sacc, Z_reel_sacc, Vol_sacc ,coef_fug_sacc, b_m):
    r = -ac_sacc + (2*np.sqrt(2)*bm*R*t*(phi-((b_sacc/bm)*(z-1))+np.log((-z+P*bm/R*t))))/np.log(1+(2*np.sqrt(2)*bm)/(vol+(1-np.sqrt(2))*bm))
    l = xe**2 *ac_sacc + xs**2 *ac_sacc + (1/(1+(b_sacc/bm)))*r
    k = 1 + (1/2*np.sqrt(ac_eau*ac_sacc))*l
    k_eau_sacc.append(k)


print('les coefficients d\'interactions binaires succharose-eau :')

data = {'Temperature': T_K, 'Fraction molaire eau': x_eau, 'Fraction molaire succharose ': x_sacc,
        'Facteur de compressibilte du succharose': Z_reel_sacc,'Facteur de compressibilte de l\'eau': Z_reel_eau,
        'ln de coefficient de fugacite de succharose' : coef_fug_sacc,
        'ln de coefficient de fugacite de l\'eau' : coef_fug_eau, 'Coefficients kij': k_eau_sacc }

df = pd.DataFrame(data)
print(df)
df.to_excel('C:\\Users\\lenovo\\Desktop\\Me\\k_eau_sacc.xlsx', index=False)




les coefficients d'interactions binaires succharose-eau :
    Temperature  Fraction molaire eau  Fraction molaire succharose   \
0        373.25              0.996948                      0.003052   
1        373.45              0.993802                      0.006198   
2        373.55              0.985797                      0.014203   
3        374.35              0.975843                      0.024157   
4        374.95              0.959529                      0.040471   
5        376.15              0.927985                      0.072015   
6        377.15              0.911002                      0.088998   
7        381.35              0.857207                      0.142793   
8        383.15              0.838880                      0.161120   
9        388.15              0.806686                      0.193314   
10       391.15              0.747913                      0.252087   
11       380.55              0.811859                      0.188141   
12       390.15    

In [18]:
import cmath
import math
import numpy as np
import matplotlib.pyplot as plt


# Constantes
M_fruc = 180.16
M_eau = 18
R = 8.314  # Constante des gaz parfaits en J/(mol·K)
# Données pour le fructose (fruc)
Tc_fruc = 884.35  # Température critique du fructode en K
Pc_fruc = 47.67*101325   # Pression critique du fructose en atm
omega_fruc = 2.210
# Données pour l'eau (eau)
Tc_eau = 647.06   # Température critique de l'eau en K
Pc_eau = 220.50*101325   # Pression critique de l'eau en atm
omega_eau = 0.348



X = np.array([0.045, 0.060, 0.095, 0.115, 0.205, 0.255, 0.340, 0.460, 0.585, 0.680, 0.820, 0.870])
x_fruc = []
for i in range(len(X)):
    frac= (X[i]/M_fruc)/(X[i]/M_fruc + (1 - X[i])/M_eau )
    x_fruc.append(frac)


x_eau = []
for j in range(len(x_fruc)):
    frac = 1 - x_fruc[j]
    x_eau.append(frac)




T_C = np.array([100.1, 100.3, 100.4, 100.6, 101.1, 101.2, 101.3, 102.5, 104.9, 108.4, 122.0, 130.0])
T_K = []
for i in T_C:
    i = i + 273.15
    T_K.append(i)

def calculate_A(T, P, a, R):
    A = (a * P) / (R * T)**2
    return A

def calculate_B(T, P, R, b):
    B = (b * P) / (R * T)
    return B

def ac(R, Tc, Pc):
    return 0.45724 * ((R**2 * Tc**2) / Pc)

def a(R, Tc, Pc, m, T):
    return ac(R, Tc, Pc) * (1 + m * (1 - np.sqrt(T/Tc)))**2

def b(R, Tc, Pc):
    return 0.07780 * (R * Tc) / Pc


def m(omega):
    return 0.37464 + 1.54226 * omega - 0.26992 * omega**2

P = 1*101325
Z_fruc = []
for i in range(len(T_K)):
    m_value = m(omega_fruc)
    ac_value = ac(R, Tc_fruc, Pc_fruc)
    a_value = a(R, Tc_fruc, Pc_fruc, m_value , T_K[i])
    b_value = b(R, Tc_fruc, Pc_fruc)
    A = calculate_A(T_K[i], P, a_value, b_value)
    B = calculate_B(T_K[i], P, a_value, b_value)
    u  = 1 - B
    v = A - 3*B**2 - 2*B
    w = A*B - B**2 - B**3
    coefficients = [1,u, v, w]
    z = np.roots(coefficients)
    Z_fruc.append(z)
Z_reel_fruc = []
for z in Z_fruc:
    z_reel = max(z, key=lambda x: x.real)
    z_reel = np.real(z_reel)
    Z_reel_fruc.append(z_reel)




Z_eau = []
for i in range(len(T_K)):
    m_value = m(omega_eau)
    ac_value = ac(R, Tc_eau, Pc_eau)
    a_value = a(R, Tc_eau, Pc_eau, m_value , T_K[i])
    b_value = b(R, Tc_eau, Pc_eau)
    A = calculate_A(T_K[i], P, a_value, b_value)
    B = calculate_B(T_K[i], P, a_value, b_value)
    u  = 1 - B
    v = A - 3*B**2 - 2*B
    w = A*B - B**2 - B**3
    coefficients = [1,u, v, w]
    z = np.roots(coefficients)
    Z_eau.append(z)
Z_reel_eau = []
for z in Z_eau:
    z_reel = max(z, key=lambda x: x.real)
    z_reel = np.real(z_reel)
    Z_reel_eau.append(z_reel)



coef_fug_eau = []
for i in range(len(Z_reel_eau)):
    a = (Z_reel_eau[i] - 1) * np.log(P)
    coef_fug_eau.append(a)

coef_fug_fruc = []
for i in range(len(Z_reel_fruc)):
    a = (Z_reel_fruc[i] - 1) * np.log(P)
    coef_fug_fruc.append(a)

Vol_eau = []
for z, t in zip(Z_reel_eau, T_K):
    v = (z * R * t)/ P
    Vol_eau.append(v)

Vol_fruc = []
for z, t in zip(Z_reel_fruc, T_K):
    v = (z * R * t)/ P
    Vol_fruc.append(v)



ac_eau = ac(R, Tc_eau, Pc_eau)
ac_fruc = ac(R, Tc_sacc, Pc_fruc)
b_eau = b(R, Tc_eau, Pc_eau)
b_fruc = b(R, Tc_fruc, Pc_fruc)

b_m = []
for xe, xs in zip(x_eau, x_fruc):
    bm = xe * b_eau + xs * b_fruc
    b_m.append(bm)




k_eau_fruc=[]
for t, xe, xs, z, vol, phi, bm in zip(T_K, x_eau, x_fruc, Z_reel_fruc, Vol_fruc ,coef_fug_fruc, b_m):
    r = -ac_fruc + (2*np.sqrt(2)*bm*R*t*(phi-((b_fruc/bm)*(z-1))+np.log((z+P*bm/R*t))))/np.log(1+(2*np.sqrt(2)*bm)/(vol+(1-np.sqrt(2))*bm))
    l = xe**2 *ac_fruc + xs**2 *ac_fruc + (1/(1+(b_fruc/bm)))*r
    k = 1 + (1/2*np.sqrt(ac_eau*ac_fruc))*l
    k_eau_fruc.append(k)


print('les coefficients d\'interactions binaires fructose-eau :')

data = {'Temperature': T_K, 'Fraction molaire eau': x_eau, 'Fraction molaire fructose ': x_fruc,
        'Facteur de compressibilte du fructose': Z_reel_fruc,'Facteur de compressibilte de l\'eau': Z_reel_eau,
        'ln de coefficient de fugacite du fructose ': coef_fug_fruc,
        'ln de coefficient de fugacite de l\'eau' : coef_fug_eau, 'Coefficients kij': k_eau_fruc}

df = pd.DataFrame(data)

print(df)
df.to_excel('C:\\Users\\lenovo\\Desktop\\Me\\k_eau_fruc.xlsx', index=False)




les coefficients d'interactions binaires fructose-eau :
    Temperature  Fraction molaire eau  Fraction molaire fructose   \
0        373.25              0.995314                    0.004686   
1        373.45              0.993663                    0.006337   
2        373.55              0.989621                    0.010379   
3        373.75              0.987184                    0.012816   
4        374.25              0.974884                    0.025116   
5        374.35              0.966933                    0.033067   
6        374.45              0.951050                    0.048950   
7        375.65              0.921566                    0.078434   
8        378.05              0.876548                    0.123452   
9        381.55              0.824871                    0.175129   
10       395.15              0.687214                    0.312786   
11       403.15              0.599292                    0.400708   

    Facteur de compressibilte du fructose  Fac

<ipython-input-18-9adcc0f7e41a>:147: RuntimeWarning: invalid value encountered in log
  r = -ac_fruc + (2*np.sqrt(2)*bm*R*t*(phi-((b_fruc/bm)*(z-1))+np.log((z+P*bm/R*t))))/np.log(1+(2*np.sqrt(2)*bm)/(vol+(1-np.sqrt(2))*bm))


In [22]:
import cmath
import cmath
import math
import numpy as np
import matplotlib.pyplot as plt


# Constantes
M_gluc = 180.156
M_eau = 18
R = 8.314  # Constante des gaz parfaits en J/(mol·K)
# Données pour le glucose (gluc)
Tc_gluc = 884.61  # Température critique du glucose en K
Pc_gluc = 47.53*101325   # Pression critique du glucose en atm
omega_gluc = 2.208
# Données pour l'eau (eau)
Tc_eau = 647.06   # Température critique de l'eau en K
Pc_eau = 220.50*101325   # Pression critique de l'eau en atm
omega_eau = 0.348



X = np.array([0.045, 0.055, 0.105, 0.12, 0.21, 0.25, 0.335, 0.41, 0.45, 0.555, 0.64, 0.87])
x_gluc = []
for i in range(len(X)):
    frac= (X[i]/M_gluc)/(X[i]/M_gluc + (1 - X[i])/M_eau )
    x_gluc.append(frac)


x_eau = []
for j in range(len(x_gluc)):
    frac = 1 - x_gluc[j]
    x_eau.append(frac)




T_C = np.array([100.2, 100.4, 100.6, 100.7, 101.3, 101.4, 101.6, 101.8, 103, 104, 106, 130])
T_K = []
for i in T_C:
    i = i + 273.15
    T_K.append(i)

def calculate_A(T, P, a, R):
    A = (a * P) / (R * T)**2
    return A

def calculate_B(T, P, R, b):
    B = (b * P) / (R * T)
    return B

def ac(R, Tc, Pc):
    return 0.45724 * ((R**2 * Tc**2) / Pc)

def a(R, Tc, Pc, m, T):
    return ac(R, Tc, Pc) * (1 + m * (1 - np.sqrt(T/Tc)))**2

def b(R, Tc, Pc):
    return 0.07780 * (R * Tc) / Pc


def m(omega):
    return 0.37464 + 1.54226 * omega - 0.26992 * omega**2



P = 1*101325
Z_gluc = []
for i in range(len(T_K)):
    m_value = m(omega_gluc)
    ac_value = ac(R, Tc_gluc, Pc_gluc)
    a_value = a(R, Tc_gluc, Pc_gluc, m_value , T_K[i])
    b_value = b(R, Tc_gluc, Pc_gluc)
    A = calculate_A(T_K[i], P, a_value, b_value)
    B = calculate_B(T_K[i], P, a_value, b_value)
    u  = 1 - B
    v = A - 3*B**2 - 2*B
    w = A*B - B**2 - B**3
    z = np.roots(coefficients)
    Z_gluc.append(z)
Z_reel_gluc = []
for z in Z_eau:
    z_reel = max(z, key=lambda x: x.real)
    z_reel = np.real(z_reel)
    Z_reel_gluc.append(z_reel)


Z_eau = []
for i in range(len(T_K)):
    m_value = m(omega_eau)
    ac_value = ac(R, Tc_eau, Pc_eau)
    a_value = a(R, Tc_eau, Pc_eau, m_value , T_K[i])
    b_value = b(R, Tc_eau, Pc_eau)
    A = calculate_A(T_K[i], P, a_value, b_value)
    B = calculate_B(T_K[i], P, a_value, b_value)
    u  = 1 - B
    v = A - 3*B**2 - 2*B
    w = A*B - B**2 - B**3
    coefficients = [1,u, v, w]
    z = np.roots(coefficients)
    Z_eau.append(z)
Z_reel_eau = []
for z in Z_eau:
    z_reel = max(z, key=lambda x: x.real)
    z_reel = np.real(z_reel)
    Z_reel_eau.append(z_reel)



coef_fug_eau = []
for i in range(len(Z_reel_eau)):
    a = (Z_reel_eau[i] - 1) * np.log(P)
    coef_fug_eau.append(a)

coef_fug_gluc = []
for i in range(len(Z_reel_gluc)):
    a = (Z_reel_gluc[i] - 1) * np.log(P)
    coef_fug_gluc.append(a)

Vol_eau = []
for z, t in zip(Z_reel_eau, T_K):
    v = (z * R * t)/ P
    Vol_eau.append(v)

Vol_gluc = []
for z, t in zip(Z_reel_gluc, T_K):
    v = (z * R * t)/ P
    Vol_gluc.append(v)



ac_eau = ac(R, Tc_eau, Pc_eau)
ac_gluc = ac(R, Tc_gluc, Pc_gluc)
b_eau = b(R, Tc_eau, Pc_eau)
b_gluc = b(R, Tc_gluc, Pc_gluc)

b_m = []
for xe, xs in zip(x_eau, x_gluc):
    bm = xe * b_eau + xs * b_gluc
    b_m.append(bm)




k_eau_gluc=[]
for t, xe, xs, z, vol, phi, bm in zip(T_K, x_eau, x_gluc, Z_reel_gluc, Vol_gluc ,coef_fug_gluc, b_m):
    r = -ac_gluc + (2*np.sqrt(2)*bm*R*t*(phi-((b_gluc/bm)*(z-1))+np.log((-z+P*bm/R*t))))/np.log(1+(2*np.sqrt(2)*bm)/(vol+(1-np.sqrt(2))*bm))
    l = xe**2 *ac_gluc + xs**2 *ac_gluc + (1/(1+(b_gluc/bm)))*r
    k = 1 + (1/2*np.sqrt(ac_eau*ac_gluc))*l
    k_eau_gluc.append(k)

print('les coefficients d\'interactions binaires glucose-eau :')

data = {'Temperature': T_K, 'Fraction molaire eau': x_eau, 'Fraction molaire glucose ': x_gluc,
        'Facteur de compressibilte du glucose': Z_reel_gluc,'Facteur de compressibilte de l\'eau': Z_reel_eau,
        'ln de coefficient de fugacite du glucose': coef_fug_gluc,
        'ln de coefficient de fugacite de l\'eau' : coef_fug_eau, 'Coefficients kij': k_eau_gluc }

df = pd.DataFrame(data)

print(df)
df.to_excel('C:\\Users\\lenovo\\Desktop\\Me\\k_eau_gluc.xlsx', index=False)




les coefficients d'interactions binaires glucose-eau :
    Temperature  Fraction molaire eau  Fraction molaire glucose   \
0        373.35              0.995314                   0.004686   
1        373.55              0.994219                   0.005781   
2        373.75              0.988414                   0.011586   
3        373.85              0.986559                   0.013441   
4        374.45              0.974128                   0.025872   
5        374.55              0.967769                   0.032231   
6        374.75              0.952080                   0.047920   
7        374.95              0.935076                   0.064924   
8        376.15              0.924430                   0.075570   
9        377.15              0.889196                   0.110804   
10       379.15              0.849168                   0.150832   
11       403.15              0.599286                   0.400714   

    Facteur de compressibilte du glucose  Facteur de compres